<font size = 6> Load Lumapi

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, Math
import sys 
import os

sys.path.append("C:\\Program Files\\Lumerical\\v202\\api\\python\\") #Default windows lumapi path
#sys.path.append("/opt/lumerical/v221/api/python/lumapi.py") #Default linux lumapi path
import lumapi

sys.path.append(os.path.dirname('C:\\Users\\andre\\Documents\\GitHub\\Photonics')) #Current directory

<font size = 6> Open Lumerical Mode

In [2]:
mmi = lumapi.MODE(filename = 'MMI.lms', hide = False)

In [34]:
Wm = 10e-6                                  #box width
L = 43.37e-6                                #box length
h = 0.22e-6                                 #total height
Wg = 1.27e-6                                #taper width
Tlen = 5e-6                                 #taper length
Tpos = [3.75e-6, 1.25e-6, -1.25e-6, -3.75e-6]    #taper position
material = 'Si (Silicon) - Palik'           #material

In [36]:
mmi.switchtolayout()
mmi.selectall()
mmi.delete()

mmi.addrect()
mmi.set({
    'x min'         : 0,
    'x max'         : L,
    'y'             : 0,
    'y span'        : Wm,
    'z'             : 0,
    'z span'        : h,
    'material'      : material,
    'name'          : 'Box'
    })

mmi.addobject('linear_taper')
mmi.set({
    'x'             : -Tlen/2,
    'y'             : 0,
    'thickness'     : h,
    'angle_side'    : 90,
    'width_l'       : 0.45e-6,
    'width_r'       : Wg,
    'len'           : Tlen,
    'material'      : material,
    'name'          : 'Taper in'
})
mmi.addtogroup('Input')

mmi.addrect()
mmi.set({
    'x min'         : -2*Tlen,
    'x max'         : -Tlen,
    'y'             : 0,
    'y span'        : 0.45e-6,
    'z'             : 0,
    'z span'        : h,
    'material'      : material,
    'name'          : 'In'
})
mmi.addtogroup('Input')

for i in range (4):
    mmi.addobject('linear_taper')
    mmi.set({
        'x'             : L + Tlen/2,
        'y'             : Tpos[i],
        'thickness'     : h,
        'angle_side'    : 90,
        'width_l'       : Wg,
        'width_r'       : 0.45e-6,
        'len'           : Tlen,
        'material'      : material,
        'name'          : 'Taper out ' + str(i+1)
    })